# 챕터 7 (OpenAI 버전): Skills로 에이전트 워크플로우 자동화 만들기

## 7-1. 에이전트 워크플로우 개요

OpenAI GPT-4o-mini를 사용하여 자율적으로 의사결정하고 작업을 수행하는 시스템을 구축한다.

**에이전트의 특징:**
- 목표 지향적: 최종 목표를 달성하기 위해 행동한다
- 자율적: 스스로 다음 단계를 결정한다
- 반응적: 환경과 상황 변화에 대응한다
- 학습 가능: 과거 경험을 활용한다

**워크플로우 패턴:**
1. ReAct (Reasoning + Acting): 추론하고 행동하기
2. Plan-Execute: 계획 수립 후 실행
3. Reflection: 결과를 평가하고 개선
4. Multi-Agent: 여러 에이전트의 협업

**OpenAI API 활용:**
- Function Calling으로 도구 사용
- JSON 모드로 구조화된 출력
- 빠른 응답으로 실시간 의사결정

In [ ]:
# 필요한 라이브러리 설치
%pip install openai pydantic python-dotenv pyyaml -q

In [2]:
from dotenv import load_dotenv
import os

MODEL = "gpt-4o-mini"
load_dotenv()

True

In [3]:
import openai
import os
from pathlib import Path
import yaml
import json
from typing import Dict, List, Optional, Any, Union, Tuple
from pydantic import BaseModel, Field
from enum import Enum
import time
from datetime import datetime
import re

# OpenAI 클라이언트 초기화
client = openai.OpenAI()

print("환경 설정 완료")
print(f"OpenAI 라이브러리 버전: {openai.__version__}")

환경 설정 완료
OpenAI 라이브러리 버전: 2.6.1


## 7-2. Pydantic 모델로 에이전트 상태 관리

에이전트의 상태와 기억을 Pydantic 모델로 관리한다.

In [4]:
class AgentState(str, Enum):
    """에이전트의 현재 상태"""
    IDLE = "idle"  # 대기
    PLANNING = "planning"  # 계획 수립
    EXECUTING = "executing"  # 실행 중
    REFLECTING = "reflecting"  # 반성 중
    COMPLETED = "completed"  # 완료
    FAILED = "failed"  # 실패

class ActionRecord(BaseModel):
    """수행한 행동 기록"""
    name: str = Field(..., description="행동 이름")
    input: Dict[str, Any] = Field(..., description="행동 입력")
    timestamp: datetime = Field(default_factory=datetime.now)

class AgentMemory(BaseModel):
    """
    에이전트의 기억과 경험
    """
    goal: str = Field(..., description="목표")
    observations: List[str] = Field(default_factory=list, description="관찰 내용")
    thoughts: List[str] = Field(default_factory=list, description="생각")
    actions: List[ActionRecord] = Field(default_factory=list, description="행동 기록")
    results: List[str] = Field(default_factory=list, description="행동 결과")
    learnings: List[str] = Field(default_factory=list, description="학습 내용")
    
    def add_observation(self, observation: str) -> None:
        """관찰 내용을 추가한다"""
        self.observations.append(observation)
    
    def add_thought(self, thought: str) -> None:
        """생각을 추가한다"""
        self.thoughts.append(thought)
    
    def add_action(self, action_name: str, action_input: Dict) -> None:
        """행동을 기록한다"""
        self.actions.append(ActionRecord(
            name=action_name,
            input=action_input
        ))
    
    def add_result(self, result: str) -> None:
        """행동 결과를 추가한다"""
        self.results.append(result)
    
    def add_learning(self, learning: str) -> None:
        """학습 내용을 추가한다"""
        self.learnings.append(learning)
    
    def get_context(self, max_items: int = 5) -> str:
        """
        현재까지의 컨텍스트를 문자열로 반환한다
        
        Args:
            max_items: 각 카테고리별 최대 아이템 수
            
        Returns:
            컨텍스트 문자열
        """
        context = f"목표: {self.goal}\n\n"
        
        if self.observations:
            context += "최근 관찰:\n"
            for i, obs in enumerate(self.observations[-max_items:], 1):
                context += f"{i}. {obs}\n"
            context += "\n"
        
        if self.thoughts:
            context += "최근 생각:\n"
            for i, thought in enumerate(self.thoughts[-3:], 1):
                context += f"{i}. {thought}\n"
            context += "\n"
        
        if self.actions:
            context += "수행한 행동:\n"
            for i, action in enumerate(self.actions[-3:], 1):
                context += f"{i}. {action.name}\n"
            context += "\n"
        
        return context

print("에이전트 상태 관리 Pydantic 모델 정의 완료")

에이전트 상태 관리 Pydantic 모델 정의 완료


## 7-3. OpenAI 기반 ReAct 에이전트 구현

추론(Reasoning)과 행동(Acting)을 반복하는 ReAct 패턴을 OpenAI API로 구현한다.

In [5]:
class ReActResponse(BaseModel):
    """ReAct 에이전트의 응답"""
    thought: str = Field(..., description="현재 상황에 대한 추론")
    action: str = Field(..., description="수행할 행동")
    action_input: str = Field(..., description="행동 입력")

class OpenAIReActAgent:
    """
    OpenAI API를 사용하는 ReAct 에이전트
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        ReAct 에이전트를 초기화한다
        
        Args:
            api_key: OpenAI API 키
        """
        self.client = openai.OpenAI()
        self.skills: Dict[str, Dict] = {}  # 사용 가능한 Skills (도구)
        self.state = AgentState.IDLE
        self.memory: Optional[AgentMemory] = None
        self.max_iterations = 10  # 최대 반복 횟수
    
    def load_skill(self, skill_path: str) -> None:
        """
        Skill을 로드하여 에이전트의 도구로 등록한다
        
        Args:
            skill_path: Skill 파일 경로
        """
        with open(skill_path, 'r', encoding='utf-8') as f:
            skill_data = yaml.safe_load(f)
        
        skill_name = skill_data['name']
        self.skills[skill_name] = skill_data
        print(f"도구 등록: {skill_name} - {skill_data['description']}")
    
    def think(self, memory: AgentMemory) -> ReActResponse:
        """
        현재 상황을 분석하고 다음 행동을 결정한다
        
        Args:
            memory: 에이전트 메모리
            
        Returns:
            ReAct 응답
        """
        # 사용 가능한 도구 목록
        tools_description = "\n".join([
            f"- {name}: {skill['description']}"
            for name, skill in self.skills.items()
        ])
        
        # ReAct 프롬프트
        system_prompt = """당신은 목표 달성을 위해 도구를 사용하는 자율 에이전트다.
각 단계에서 현재 상황을 분석하고, 다음에 수행할 행동을 결정한다.

응답 형식:
생각: [현재 상황 분석과 다음 행동에 대한 추론]
행동: [사용할 도구 이름]
입력: [도구에 전달할 입력 내용]

목표를 달성했다면:
생각: [목표 달성 이유]
행동: FINISH
입력: [최종 답변]
"""
        
        user_prompt = f"""{memory.get_context()}

사용 가능한 도구:
{tools_description}

다음 행동을 결정해줘.
"""
        
        response = self.client.chat.completions.create(
            model=MODEL,
            max_tokens=1000,
            temperature=0.7,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        
        response_text = response.choices[0].message.content
        
        # 응답 파싱
        thought_match = re.search(r'생각:\s*(.+?)(?=\n행동:|$)', response_text, re.DOTALL)
        action_match = re.search(r'행동:\s*(.+?)(?=\n입력:|$)', response_text)
        input_match = re.search(r'입력:\s*(.+?)(?=\n|$)', response_text, re.DOTALL)
        
        thought = thought_match.group(1).strip() if thought_match else "생각 파싱 실패"
        action = action_match.group(1).strip() if action_match else "FINISH"
        action_input = input_match.group(1).strip() if input_match else ""
        
        return ReActResponse(
            thought=thought,
            action=action,
            action_input=action_input
        )
    
    def act(self, action_name: str, action_input: str) -> str:
        """
        행동을 수행한다
        
        Args:
            action_name: 수행할 행동 (Skill) 이름
            action_input: 행동 입력
            
        Returns:
            행동 결과
        """
        if action_name not in self.skills:
            return f"오류: '{action_name}' 도구를 찾을 수 없다"
        
        skill = self.skills[action_name]
        config = skill.get('config', {})
        
        response = self.client.chat.completions.create(
            model=config.get('model', MODEL),
            max_tokens=config.get('max_tokens', 2000),
            temperature=config.get('temperature', 0.7),
            messages=[
                {"role": "system", "content": skill['content']},
                {"role": "user", "content": action_input}
            ]
        )
        
        return response.choices[0].message.content
    
    def run(self, goal: str, initial_observation: str = "") -> str:
        """
        에이전트를 실행하여 목표를 달성한다
        
        Args:
            goal: 달성할 목표
            initial_observation: 초기 관찰 내용
            
        Returns:
            최종 결과
        """
        self.memory = AgentMemory(goal=goal)
        self.state = AgentState.EXECUTING
        
        if initial_observation:
            self.memory.add_observation(initial_observation)
        
        print("\n" + "=" * 80)
        print(f"OpenAI ReAct 에이전트 실행 시작")
        print(f"목표: {goal}")
        print(f"모델: gpt-4o-mini")
        print("=" * 80)
        
        for iteration in range(1, self.max_iterations + 1):
            print(f"\n[반복 {iteration}/{self.max_iterations}]")
            
            # 1. 생각하기 (Reasoning)
            react_response = self.think(self.memory)
            self.memory.add_thought(react_response.thought)
            
            print(f"생각: {react_response.thought[:200]}..." if len(react_response.thought) > 200 else f"생각: {react_response.thought}")
            print(f"행동: {react_response.action}")
            
            # 2. 완료 확인
            if react_response.action == "FINISH":
                final_answer = react_response.action_input
                print(f"\n최종 답변: {final_answer}")
                self.state = AgentState.COMPLETED
                return final_answer
            
            # 3. 행동하기 (Acting)
            self.memory.add_action(react_response.action, {"content": react_response.action_input})
            result = self.act(react_response.action, react_response.action_input)
            self.memory.add_result(result)
            self.memory.add_observation(f"{react_response.action} 실행 결과: {result[:150]}...")
            
            print(f"결과: {result[:200]}..." if len(result) > 200 else f"결과: {result}")
        
        # 최대 반복 횟수 초과
        print(f"\n최대 반복 횟수({self.max_iterations})에 도달했다")
        self.state = AgentState.FAILED
        return "목표를 달성하지 못했다"

print("OpenAIReActAgent 클래스 정의 완료")

OpenAIReActAgent 클래스 정의 완료


## 7-4. 에이전트용 Skills 생성

In [8]:
# skills 디렉토리 생성
skills_dir = Path("skills/agent_workflow_openai")
skills_dir.mkdir(exist_ok=True)

# 1. 웹 검색 시뮬레이터 Skill
web_search_skill = {
    'name': 'web_search',
    'version': '1.0.0',
    'description': '인터넷에서 정보를 검색한다',
    'content': """# 웹 검색 도구 Skill

## 역할
사용자의 검색 쿼리에 대해 관련 정보를 제공하는 웹 검색 엔진이다.

## 검색 원칙
1. 정확성: 신뢰할 수 있는 정보를 제공한다
2. 최신성: 가능한 최신 정보를 우선한다
3. 다양성: 여러 관점의 정보를 포함한다

## 출력 형식
- 검색 결과 3-5개를 제공한다
- 각 결과는 제목, 요약, 출처를 포함한다

## 주의사항
실제 웹 검색이 아닌 시뮬레이션이므로, 일반적이고 합리적인 정보를 생성한다.

## 스타일
- 문장은 ~다 체를 사용한다
- 객관적이고 정보 전달에 초점을 맞춘다
""",
    'config': {
        'model': MODEL,
        'max_tokens': 1500,
        'temperature': 0.5
    }
}

# 2. 계산기 Skill
calculator_skill = {
    'name': 'calculator',
    'version': '1.0.0',
    'description': '수학 계산을 수행한다',
    'content': """# 계산기 도구 Skill

## 역할
수학적 계산과 분석을 수행하는 계산기다.

## 기능
1. 기본 사칙연산
2. 복잡한 수식 계산
3. 통계 계산
4. 단위 변환

## 출력 형식
- 계산 과정을 단계별로 보여준다
- 최종 결과를 명확히 표시한다

## 스타일
- 문장은 ~다 체를 사용한다
- 정확하고 명확한 답변을 제공한다
""",
    'config': {
        'model': MODEL,
        'max_tokens': 1000,
        'temperature': 0.1
    }
}

# 3. 텍스트 분석 Skill
text_analyzer_skill = {
    'name': 'text_analyzer',
    'version': '1.0.0',
    'description': '텍스트를 분석하고 인사이트를 도출한다',
    'content': """# 텍스트 분석 도구 Skill

## 역할
텍스트를 다양한 관점에서 분석하는 전문가다.

## 분석 항목
1. 감정 분석: 긍정/부정/중립
2. 주제 추출: 핵심 주제와 키워드
3. 문체 분석: 톤, 스타일, 난이도
4. 구조 분석: 논리적 구조와 흐름

## 출력 형식
- 각 분석 항목별로 결과를 제시한다
- 구체적인 예시를 포함한다

## 스타일
- 문장은 ~다 체를 사용한다
- 분석적이고 객관적인 언어를 사용한다
""",
    'config': {
        'model': MODEL,
        'max_tokens': 2000,
        'temperature': 0.4
    }
}

# 4. 코드 생성 Skill
code_generator_skill = {
    'name': 'code_generator',
    'version': '1.0.0',
    'description': '프로그래밍 코드를 생성한다',
    'content': """# 코드 생성 도구 Skill

## 역할
요구사항에 맞는 프로그래밍 코드를 생성하는 전문가다.

## 코드 생성 원칙
1. 가독성: 이해하기 쉬운 코드를 작성한다
2. 효율성: 최적화된 알고리즘을 사용한다
3. 안정성: 에러 처리를 포함한다
4. 주석: 복잡한 로직에는 설명을 추가한다

## 지원 언어
Python, JavaScript, Java, C++, Go 등

## 출력 형식
- 완전히 동작하는 코드를 제공한다
- 사용 방법과 예제를 포함한다

## 스타일
- 문장은 ~다 체를 사용한다
- 기술적으로 정확한 설명을 제공한다
""",
    'config': {
        'model': MODEL,
        'max_tokens': 3000,
        'temperature': 0.6
    }
}

# Skills 저장
agent_skills = [
    web_search_skill,
    calculator_skill,
    text_analyzer_skill,
    code_generator_skill
]

for skill in agent_skills:
    skill_path = skills_dir / f"{skill['name']}.yaml"
    with open(skill_path, 'w', encoding='utf-8') as f:
        yaml.dump(skill, f, allow_unicode=True, default_flow_style=False)
    print(f"Skill 저장: {skill_path}")

print(f"\n총 {len(agent_skills)}개의 에이전트 Skill이 생성되었다")

Skill 저장: skills/agent_workflow_openai/web_search.yaml
Skill 저장: skills/agent_workflow_openai/calculator.yaml
Skill 저장: skills/agent_workflow_openai/text_analyzer.yaml
Skill 저장: skills/agent_workflow_openai/code_generator.yaml

총 4개의 에이전트 Skill이 생성되었다


## 7-5. 예제 1: 리서치 에이전트

주제를 조사하고 분석하는 자율 에이전트다.

In [9]:
# ReAct 에이전트 생성
research_agent = OpenAIReActAgent()

# 도구 로드
research_agent.load_skill('skills/agent_workflow_openai/web_search.yaml')
research_agent.load_skill('skills/agent_workflow_openai/text_analyzer.yaml')
research_agent.load_skill('skills/agent_workflow_openai/calculator.yaml')

print("\n리서치 에이전트 준비 완료")

도구 등록: web_search - 인터넷에서 정보를 검색한다
도구 등록: text_analyzer - 텍스트를 분석하고 인사이트를 도출한다
도구 등록: calculator - 수학 계산을 수행한다

리서치 에이전트 준비 완료


In [10]:
# 에이전트 실행
goal = """
'전기차 시장 성장률'에 대해 조사하고, 
다음 정보를 포함한 종합 보고서를 작성해줘:
1. 현재 시장 규모
2. 연평균 성장률 (CAGR)
3. 주요 시장 (국가별)
4. 향후 5년 전망
"""

result = research_agent.run(
    goal=goal,
    initial_observation="전기차 시장에 대한 정보가 필요하다"
)

print("\n" + "=" * 80)
print("최종 결과")
print("=" * 80)
print(result)


OpenAI ReAct 에이전트 실행 시작
목표: 
'전기차 시장 성장률'에 대해 조사하고, 
다음 정보를 포함한 종합 보고서를 작성해줘:
1. 현재 시장 규모
2. 연평균 성장률 (CAGR)
3. 주요 시장 (국가별)
4. 향후 5년 전망

모델: gpt-4o-mini

[반복 1/10]
생각: 현재 전기차 시장에 대한 정보를 수집해야 하며, 필요한 세부사항(현재 시장 규모, CAGR, 주요 시장, 향후 5년 전망)을 포함한 종합 보고서를 작성하기 위해 인터넷 검색 도구를 사용할 것이다.
행동: web_search
결과: 다음은 2023년 전기차 시장에 대한 정보와 향후 전망이다.

1. **전기차 시장 규모 및 성장률**
   - 2023년 전기차 시장 규모는 약 1,200억 달러로 추정되며, 2022년 대비 약 25% 성장하였다. 
   - CAGR(연평균 성장률)은 2023년부터 2028년까지 약 20%에 이를 것으로 예상된다.
   - 출처: MarketsandMar...

[반복 2/10]
생각: 현재 전기차 시장에 대한 정보가 부족하며, 특히 연평균 성장률(CAGR)과 주요 시장(국가별), 향후 5년 전망에 대한 정보를 추가로 수집해야 한다. 이를 위해 다시 인터넷 검색 도구를 사용할 필요가 있다.
행동: web_search
결과: 전기차 시장에 대한 2023년 CAGR 및 주요 국가별 시장 전망에 대한 정보를 제공한다.

1. **전기차 시장, 2023년 CAGR 20% 이상 예상**
   - 2023년 전기차 시장의 연평균 성장률(CAGR)은 20% 이상으로 예상되며, 이는 전 세계적으로 전기차 수요가 급증하고 있음을 반영한다. 특히, 북미와 유럽 시장이 주요 성장 동력으로 작용할...

[반복 3/10]
생각: 현재 전기차 시장에 대한 정보를 수집하였으나, 연평균 성장률(CAGR)과 주요 시장(국가별), 향후 5년 전망에 대한 정보가 부족하다. 따라서 추가적인 세부사항을 확보하기 위해 인터넷 검색 도구인 web_search를 다시 사용할 필

## 7-6. OpenAI 기반 Plan-Execute 에이전트

먼저 전체 계획을 수립한 후 단계별로 실행하는 에이전트다.

In [11]:
class PlanStep(BaseModel):
    """계획의 단일 단계"""
    description: str = Field(..., description="단계 설명")
    tool: str = Field(..., description="사용할 도구")
    input_text: str = Field(..., description="도구 입력")

class ExecutionResult(BaseModel):
    """실행 결과"""
    step: PlanStep
    result: str
    timestamp: datetime = Field(default_factory=datetime.now)

class OpenAIPlanExecuteAgent:
    """
    OpenAI API를 사용하는 Plan-Execute 에이전트
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Plan-Execute 에이전트를 초기화한다
        """
        self.client = openai.OpenAI(
            api_key=api_key or os.environ.get("OPENAI_API_KEY")
        )
        self.skills: Dict[str, Dict] = {}
        self.plan: List[PlanStep] = []  # 실행 계획
    
    def load_skill(self, skill_path: str) -> None:
        """Skill을 로드한다"""
        with open(skill_path, 'r', encoding='utf-8') as f:
            skill_data = yaml.safe_load(f)
        
        skill_name = skill_data['name']
        self.skills[skill_name] = skill_data
        print(f"도구 등록: {skill_name} - {skill_data['description']}")
    
    def create_plan(self, goal: str) -> List[PlanStep]:
        """
        목표 달성을 위한 계획을 수립한다
        
        Args:
            goal: 달성할 목표
            
        Returns:
            단계별 계획 리스트
        """
        tools_description = "\n".join([
            f"- {name}: {skill['description']}"
            for name, skill in self.skills.items()
        ])
        
        system_prompt = """당신은 목표를 단계별 계획으로 분해하는 전략가다.
각 단계는 명확하고 실행 가능해야 한다."""
        
        user_prompt = f"""
다음 목표를 달성하기 위한 단계별 계획을 수립해줘:

목표: {goal}

사용 가능한 도구:
{tools_description}

다음 형식으로 3-5단계의 계획을 작성해줘:

1. [단계 설명]
   도구: [사용할 도구 이름]
   입력: [도구에 전달할 내용]

2. [단계 설명]
   도구: [사용할 도구 이름]
   입력: [도구에 전달할 내용]

...
"""
        
        response = self.client.chat.completions.create(
            model=MODEL,
            max_tokens=2000,
            temperature=0.7,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        
        plan_text = response.choices[0].message.content
        
        # 계획 파싱
        steps = []
        step_blocks = re.findall(
            r'\d+\.\s*(.+?)\s*도구:\s*(.+?)\s*입력:\s*(.+?)(?=\n\d+\.|$)',
            plan_text,
            re.DOTALL
        )
        
        for description, tool, input_text in step_blocks:
            steps.append(PlanStep(
                description=description.strip(),
                tool=tool.strip(),
                input_text=input_text.strip()
            ))
        
        return steps
    
    def execute_step(self, step: PlanStep) -> str:
        """
        단일 단계를 실행한다
        
        Args:
            step: 실행할 단계
            
        Returns:
            실행 결과
        """
        tool_name = step.tool
        
        if tool_name not in self.skills:
            return f"오류: '{tool_name}' 도구를 찾을 수 없다"
        
        skill = self.skills[tool_name]
        config = skill.get('config', {})
        
        response = self.client.chat.completions.create(
            model=config.get('model', 'gpt-4o-mini'),
            max_tokens=config.get('max_tokens', 2000),
            temperature=config.get('temperature', 0.7),
            messages=[
                {"role": "system", "content": skill['content']},
                {"role": "user", "content": step.input_text}
            ]
        )
        
        return response.choices[0].message.content
    
    def run(self, goal: str) -> Dict[str, Any]:
        """
        에이전트를 실행한다
        
        Args:
            goal: 달성할 목표
            
        Returns:
            실행 결과 딕셔너리
        """
        print("\n" + "=" * 80)
        print("OpenAI Plan-Execute 에이전트 실행")
        print("=" * 80)
        print(f"\n목표: {goal}")
        
        # 1단계: 계획 수립
        print("\n[1단계: 계획 수립]")
        self.plan = self.create_plan(goal)
        
        print(f"\n수립된 계획 ({len(self.plan)}단계):")
        for i, step in enumerate(self.plan, 1):
            print(f"\n{i}. {step.description}")
            print(f"   도구: {step.tool}")
        
        # 2단계: 실행
        print("\n" + "=" * 80)
        print("[2단계: 계획 실행]")
        print("=" * 80)
        
        results = []
        for i, step in enumerate(self.plan, 1):
            print(f"\n[{i}/{len(self.plan)}] {step.description}")
            result = self.execute_step(step)
            execution_result = ExecutionResult(step=step, result=result)
            results.append(execution_result)
            print(f"완료 (결과 길이: {len(result)} 문자)")
        
        print("\n" + "=" * 80)
        print("계획 실행 완료")
        print("=" * 80)
        
        return {
            'goal': goal,
            'plan': [step.model_dump() for step in self.plan],
            'results': [{
                'step': r.step.description,
                'result': r.result
            } for r in results]
        }

print("OpenAIPlanExecuteAgent 클래스 정의 완료")

OpenAIPlanExecuteAgent 클래스 정의 완료


## 7-7. 예제 2: 코딩 어시스턴트 에이전트

In [12]:
# Plan-Execute 에이전트 생성
coding_agent = OpenAIPlanExecuteAgent()

# 도구 로드
coding_agent.load_skill('skills/agent_workflow_openai/web_search.yaml')
coding_agent.load_skill('skills/agent_workflow_openai/code_generator.yaml')
coding_agent.load_skill('skills/agent_workflow_openai/text_analyzer.yaml')

print("\n코딩 어시스턴트 에이전트 준비 완료")

도구 등록: web_search - 인터넷에서 정보를 검색한다
도구 등록: code_generator - 프로그래밍 코드를 생성한다
도구 등록: text_analyzer - 텍스트를 분석하고 인사이트를 도출한다

코딩 어시스턴트 에이전트 준비 완료


In [13]:
# 에이전트 실행
coding_goal = """
Python으로 간단한 할 일 관리 CLI 프로그램을 만들어줘.
기능:
1. 할 일 추가
2. 할 일 목록 보기
3. 할 일 완료 표시
4. 할 일 삭제

JSON 파일로 데이터를 저장하고, argparse를 사용해야 한다.
"""

coding_result = coding_agent.run(coding_goal)

# 최종 결과 출력
print("\n" + "=" * 80)
print("최종 생성된 코드")
print("=" * 80)
if coding_result['results']:
    final_code = coding_result['results'][-1]['result']
    print(final_code[:1000] + "..." if len(final_code) > 1000 else final_code)


OpenAI Plan-Execute 에이전트 실행

목표: 
Python으로 간단한 할 일 관리 CLI 프로그램을 만들어줘.
기능:
1. 할 일 추가
2. 할 일 목록 보기
3. 할 일 완료 표시
4. 할 일 삭제

JSON 파일로 데이터를 저장하고, argparse를 사용해야 한다.


[1단계: 계획 수립]

수립된 계획 (4단계):

1. **프로그램 구조 설계**
   도구: text_analyzer

2. **기능별 코드 생성**
   도구: code_generator

3. **테스트 및 디버깅**
   도구: code_generator

4. **문서화 및 사용법 작성**
   도구: text_analyzer

[2단계: 계획 실행]

[1/4] **프로그램 구조 설계**
완료 (결과 길이: 735 문자)

[2/4] **기능별 코드 생성**
완료 (결과 길이: 2638 문자)

[3/4] **테스트 및 디버깅**
완료 (결과 길이: 1624 문자)

[4/4] **문서화 및 사용법 작성**
완료 (결과 길이: 626 문자)

계획 실행 완료

최종 생성된 코드
주어진 요청에 대해 텍스트 분석을 진행하겠다.

### 1. 감정 분석
- **결과**: 중립
- **설명**: 요청 내용은 특정한 감정을 나타내지 않으며, 단순히 정보 제공을 요구하는 중립적인 문장이다.

### 2. 주제 추출
- **핵심 주제**: CLI 프로그램 사용법 문서화
- **키워드**: CLI 프로그램, 사용법, 문서화, 기능 설명
- **설명**: 요청의 주제는 CLI(명령줄 인터페이스) 프로그램의 사용법을 문서화하는 것이며, 각 기능에 대한 설명을 포함해야 한다.

### 3. 문체 분석
- **톤**: 중립적이며 요청적인 톤
- **스타일**: 간결하고 명확한 표현
- **난이도**: 낮음
- **설명**: 문장은 명확하고 간결하게 작성되어 있으며, 복잡한 용어나 문장이 사용되지 않아 이해하기 쉽다.

### 4. 구조 분석
- **논리적 구조**: 요청의 구조는 명

## 7-8. 워크플로우 결과 저장 및 분석

In [14]:
def save_agent_results(
    agent_type: str,
    results: Union[str, Dict],
    memory: Optional[AgentMemory] = None,
    output_dir: str = "agent_outputs_openai"
) -> None:
    """
    에이전트 실행 결과를 저장한다
    
    Args:
        agent_type: 에이전트 타입 (react, plan_execute 등)
        results: 실행 결과
        memory: 에이전트 메모리 (선택)
        output_dir: 출력 디렉토리
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = output_path / f"{agent_type}_{timestamp}"
    run_dir.mkdir(exist_ok=True)
    
    # 결과 저장
    if isinstance(results, dict):
        # Plan-Execute 결과
        with open(run_dir / "results.json", 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2, default=str)
        
        # 마크다운 보고서
        with open(run_dir / "report.md", 'w', encoding='utf-8') as f:
            f.write(f"# {agent_type.upper()} 에이전트 실행 보고서\n\n")
            f.write(f"실행 시간: {timestamp}\n")
            f.write(f"모델: gpt-4o-mini\n\n")
            f.write(f"## 목표\n{results['goal']}\n\n")
            f.write(f"## 계획\n")
            for i, step in enumerate(results['plan'], 1):
                f.write(f"{i}. {step['description']}\n")
            f.write(f"\n## 실행 결과\n")
            for i, result in enumerate(results['results'], 1):
                f.write(f"\n### 단계 {i}: {result['step']}\n")
                f.write(f"{result['result']}\n")
    else:
        # ReAct 결과
        with open(run_dir / "result.txt", 'w', encoding='utf-8') as f:
            f.write(results)
    
    # 메모리 저장 (Pydantic 모델)
    if memory:
        with open(run_dir / "memory.json", 'w', encoding='utf-8') as f:
            json.dump(memory.model_dump(mode='json'), f, ensure_ascii=False, indent=2, default=str)
    
    print(f"\n결과 저장 완료: {run_dir}")

# 결과 저장
if research_agent.memory:
    save_agent_results(
        agent_type="react_research",
        results=result,
        memory=research_agent.memory
    )

save_agent_results(
    agent_type="plan_execute_coding",
    results=coding_result
)


결과 저장 완료: agent_outputs_openai/react_research_20251108_180019

결과 저장 완료: agent_outputs_openai/plan_execute_coding_20251108_180019


## 7-9. 생성된 파일 목록

In [15]:
print("\n" + "=" * 80)
print("생성된 모든 파일")
print("=" * 80)

# Skills
print("\n[에이전트 Skills]")
for file_path in sorted(skills_dir.glob("*.yaml")):
    size = file_path.stat().st_size
    print(f"  {file_path.name} ({size:,} bytes)")

# 에이전트 출력
agent_output_dir = Path("agent_outputs_openai")
if agent_output_dir.exists():
    print("\n[에이전트 실행 결과]")
    for run_dir in sorted(agent_output_dir.glob("*")):
        print(f"  {run_dir.name}/")
        for file_path in sorted(run_dir.glob("*")):
            size = file_path.stat().st_size
            print(f"    {file_path.name} ({size:,} bytes)")


생성된 모든 파일

[에이전트 Skills]
  calculator.yaml (588 bytes)
  code_generator.yaml (836 bytes)
  text_analyzer.yaml (731 bytes)
  web_search.yaml (863 bytes)

[에이전트 실행 결과]
  plan_execute_coding_20251108_180019/
    report.md (9,695 bytes)
    results.json (10,928 bytes)
  react_research_20251108_180019/
    memory.json (28,795 bytes)
    result.txt (32 bytes)
